In [24]:
import cmocean.cm as cmo
from satpy import Scene, find_files_and_readers
from pyresample import create_area_def
from satpy.writers import get_enhanced_image
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob
from satpy import MultiScene
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import xarray as xr
import geopy.distance
from scipy import spatial
from geopy.distance import great_circle 

import netCDF4 as nc
from csv import writer
from statistics import mean


In [16]:
eachfile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV/sprectra_pchip_baseline_corrected_WC.csv'
data = pd.read_csv(eachfile)
rawsampleIDlist = data['Sample_ID'].tolist()
sampleIDlist = []

for sample in rawsampleIDlist:
    if sample not in sampleIDlist:
        sampleIDlist.append(int(sample))

cruisetime = 120000
cruisedatelist = []
stationlist = []
depthlist = []
cruiseDOC = []
for sampleID in sampleIDlist:
    metadatafile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/TOC_GOM.csv'
    readmetadatafile = pd.read_csv(metadatafile)
    samplerow = readmetadatafile.loc[readmetadatafile['Sample ID'] == sampleID]
    try:
        cruisedatelist.append(samplerow['Date'].tolist()[0][0:4] + samplerow['Date'].tolist()[0][5:7] + samplerow['Date'].tolist()[0][8:410])
        stationlist.append(samplerow['Location'].tolist()[0].split('St.')[1])
        depthlist.append(samplerow['Depth'].tolist()[0])
        cruiseDOC.append(samplerow['NPOC_uM_std'].tolist()[0])
    except:
        pass

cruiseIDlist = []
deletedvaluelist = []
for date in cruisedatelist:
    if int(date) >= 20210715 and int(date) <= 20210727:
        cruiseIDlist.append('SU21')
    elif int(date) >= 20211022 and int(date) <= 20211113:
        cruiseIDlist.append('FA21')
    elif int(date) >= 20220409 and int(date) <= 20220424:
        cruiseIDlist.append('SP22')
    elif int(date) >= 20220706 and int(date) <= 20220728:
        cruiseIDlist.append('SU22')

for i, stationname in enumerate(stationlist):
    cruisename = cruiseIDlist[i][2:4] + cruiseIDlist[i][0:2]
    currentsample = sampleIDlist[i]
    print(cruisename)
    cruisefiles = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV Old/CDOM_outputs_*_GOM' + cruisename + '_St.' + stationname + '.csv')
    
    specslopefile = pd.read_csv(cruisefiles[0])
    alldatarows = specslopefile.loc[specslopefile['Sample_ID'] == currentsample]
    print(alldatarows['ss_275_295_sem'].tolist()[0])
    
    
    
    
    
    
    

21SU


IndexError: list index out of range

In [3]:
# To pull CTD spectral slope and DOC

files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV Old/CDOM_outputs*.csv')
files.remove('/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV Old/CDOM_outputs_master.csv')

for file in files:
    with open(file, 'r') as reader:
        with open('/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV Old/CDOM_outputs_master.csv', 'a') as writer:
            for line in reader:
                print(line)
                writer.write(line)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
eachfile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV/sprectra_pchip_baseline_corrected_WC.csv'
data = pd.read_csv(eachfile)
rawsampleIDlist = data['Sample_ID'].tolist()
sampleIDlist = []

for sample in rawsampleIDlist:
    if sample not in sampleIDlist:
        sampleIDlist.append(int(sample))
        
cruiseSpecSlope = []
for sampleID in sampleIDlist:
    SSdatafile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/CDOM_outputs_master.csv'
    readSSdatafile = pd.read_csv(SSdatafile)
    SSsamplerow = readSSdatafile.loc[readSSdatafile['Sample_ID'] == sampleID]
    try:
        cruiseSpecSlope.append(mean(SSsamplerow['ss_275_295_sem'].tolist())) 
        
    except:
        pass
print(cruiseSpecSlope)

[0.174594228482589, 0.217734723553486, 0.26936807844115934, 0.25353406511145465, 0.14586082646365667, 0.13604428940092766, 0.10192981026055067, 0.099864918810306, 0.10838934275515567, 0.096672483813571, 0.040765701734672, 0.029511860790522, 0.04402412216826, 0.038828563160563, 0.032503261909627, 0.030243686211726, 0.039930807626105, 0.03776845521218, 0.029116885195666, 0.034857818804515495, 0.026347277765922, 0.046219208188968, 0.049924726092901, 0.053971101514578, 0.042309061571721, 0.026726835163594, 0.033680422843739, 0.032231979568498, 0.014741712672002, 0.006161015215205, 0.021734925821896, 0.043488546073883, 0.037398675003157, 0.011266677260116, 0.030169354329214, 0.01845247272308, 0.034067484766533, 0.039562811214318, 0.010134822817893, 0.005660400026211001, 0.010401726871741, 0.006571725200501, 0.005964352190387, 0.007153513718306, 0.010603457621811, 0.01472978145544, 7.09172699891178e-05, 2.83789116983535e-05, 4.3618349394458205e-05, 0.005052602169743566, 7.10207541475938e-05,

In [30]:
#Working code for surface water
eachfile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV/sprectra_pchip_baseline_corrected_WC.csv'
data = pd.read_csv(eachfile)
rawsampleIDlist = data['Sample_ID'].tolist()
sampleIDlist = []

for sample in rawsampleIDlist:
    if sample not in sampleIDlist:
        sampleIDlist.append(int(sample))

cruisetime = 120000
cruisedatelist = []
stationlist = []
depthlist = []
cruiseDOC = []
cruiseSpecSlope = []
for sampleID in sampleIDlist:
    metadatafile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/TOC_GOM.csv'
    readmetadatafile = pd.read_csv(metadatafile)
    samplerow = readmetadatafile.loc[readmetadatafile['Sample ID'] == sampleID]
    try:
        cruisedatelist.append(samplerow['Date'].tolist()[0][0:4] + samplerow['Date'].tolist()[0][5:7] + samplerow['Date'].tolist()[0][8:410])
        stationlist.append(samplerow['Location'].tolist()[0].split('St.')[1])
        depthlist.append(samplerow['Depth'].tolist()[0])
        cruiseDOC.append(samplerow['NPOC_uM'].tolist()[0])

        
    except:
        pass
    
    SSdatafile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/CDOM_outputs_master.csv'
    readSSdatafile = pd.read_csv(SSdatafile)
    SSsamplerow = readSSdatafile.loc[readSSdatafile['Sample_ID'] == sampleID]
    try:
        cruiseSpecSlope.append(mean(SSsamplerow['ss_275_295_sem'].tolist())) 
        
    except:
        pass

cruiseIDlist = []
deletedvaluelist = []
for date in cruisedatelist:
    if int(date) >= 20210715 and int(date) <= 20210727:
        cruiseIDlist.append('SU21')
    elif int(date) >= 20211022 and int(date) <= 20211113:
        cruiseIDlist.append('FA21')
    elif int(date) >= 20220409 and int(date) <= 20220424:
        cruiseIDlist.append('SP22')
    elif int(date) >= 20220706 and int(date) <= 20220728:
        cruiseIDlist.append('SU22')

cruiselats = []
cruiselons = []
for i, cruise in enumerate(cruiseIDlist):
    cruisedatafile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Cruise Station Python Data.csv'
    readcruisedatafile = pd.read_csv(cruisedatafile)
    allstationrows = readcruisedatafile.loc[readcruisedatafile['CruiseID'] == cruise]
    specificstationrow = allstationrows.loc[allstationrows['Station'] == stationlist[i]]
    cruiselats.append(specificstationrow['lat'].tolist()[0])
    cruiselons.append(specificstationrow['lon'].tolist()[0])

cruiseaCDOM300 = []
cruiseaCDOM355 = []
for sample in sampleIDlist:
    eachfile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Surface CSV/sprectra_pchip_baseline_corrected_WC.csv'
    data = pd.read_csv(eachfile)
    alldatarows = data.loc[data['Sample_ID'] == sample]
    data300rows = alldatarows['300'].tolist()
    data355rows = alldatarows['355'].tolist()
    cruiseaCDOM300.append(statistics.mean(data300rows))
    cruiseaCDOM355.append(statistics.mean(data355rows))



RSFilelist = []
datetimedifflist = []
for cruisedate in cruisedatelist:
    if cruisedate <= '20220416' and cruisedate >= '20220401':
        RSFiles = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/20220417T185501Adjusted_MODIS_L2.csv.csv')
    else:
        RSFiles = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+ cruisedate +'*.csv')
    timedifflist = []
    for rfile in RSFiles:
        RSFiletime = rfile.split('/')[7][9:15]
        RSFiledate = rfile.split('/')[7][0:8]
        RSdatetime = int(str(RSFiledate) + str(RSFiletime))
        cruisedatetime = int(str(cruisedate) + str(cruisetime))
        datetimediff = abs(RSdatetime - cruisedatetime)
        timedifflist.append(datetimediff)
    
    ind = timedifflist.index(min(timedifflist))
    datetimedifflist.append(timedifflist[ind])
    RSFilelist.append(RSFiles[ind])
        
RSFilelons = []
RSFilelats = []
distancelist = []
RSaCDOM300list = []
RSaCDOM355list = []
RSDOClist = []
RSS275295list = []
for i, RSFile in enumerate(RSFilelist):
    data = pd.read_csv(RSFile)
    RSlat = data['lat'].tolist()
    RSlon = data['lon'].tolist()
    RSaCDOM300 = data['aCDOM300'].tolist()
    RSaCDOM355 = data['aCDOM355'].tolist()
    RSDOC = data['DOC'].tolist()
    RSS275295 = data['S275_295'].tolist()
    Cruisecoords = (cruiselats[i], cruiselons[i])
    distances = []
    for RScoords in zip(RSlat, RSlon):
        distances.append(abs(great_circle(Cruisecoords, RScoords).km))
    ind = distances.index(min(distances))
    distancelist.append(min(distances))
    RSFilelons.append(RSlon[ind])
    RSFilelats.append(RSlat[ind])
    RSaCDOM300list.append(RSaCDOM300[ind])
    RSaCDOM355list.append(RSaCDOM355[ind])
    RSDOClist.append(RSDOC[ind])
    RSS275295list.append(RSS275295[ind])
 
    
top = len(RSaCDOM355list)
i = 0
while i <= top:

    rowtoadd = [sampleIDlist[i], cruisedatelist[i], cruiseIDlist[i], stationlist[i], cruiselats[i], 
                cruiselons[i], depthlist[i], cruiseaCDOM300[i], cruiseaCDOM355[i], cruiseDOC[i], 
                cruiseSpecSlope[i], RSFilelist[i], RSFilelats[i], RSFilelons[i], 
                RSaCDOM300list[i], RSaCDOM355list[i], RSDOClist[i], RSS275295list[i], datetimedifflist[i], 
                distancelist[i], float(RSaCDOM300list[i])-float(cruiseaCDOM300[i]), 
                float(RSaCDOM355list[i])-float(cruiseaCDOM355[i])]
    with open('/home/hboi-ouri/Projects/NASA_ProjectExp/outputs/Tasks/Task8/Validation Table Surface Water Redo.csv', 'a') as file:
        writer_object = writer(file)
        writer_object.writerow(rowtoadd)
        file.close()

    i = i+1
    

IndexError: list index out of range

In [4]:
latlonfile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/Cruise Station Python Data.csv'
latlondata = pd.read_csv(latlonfile)
stationname = eachfile.split('/')[8].split('.')[1].split('_')[0].split('-')[0].split(' ')[0]
cruiseID = cruisefile[21:23]+ cruisefile[19:21] 
if cruiseID == '2120':
    cruiseID = 'SU21'
allstationrows = latlondata.loc[latlondata['CruiseID'] == cruiseID]
specificstationrow = allstationrows.loc[allstationrows['Station'] == stationname]

sampleIDlist = data['Sample_ID'].tolist()

for sample in sampleIDlist:
    metadatafile = '/home/hboi-ouri/Projects/RS_Files/Task8/Surface Water Data/meta_data.csv'
    metadatafile = pd.read_csv(metadatafile)
    samplerow = metadatafile.loc[metadatafile['Sample'] == sample]
    month = samplerow['Date'].tolist()[0].split('/')[0]
    if len(str(month)) < 2:
        month = '0' + month
    day = samplerow['Date'].tolist()[0].split('/')[1]
    if len(str(day)) < 2:
        day = '0' + day
    year = samplerow['Date'].tolist()[0].split('/')[2]
cruisedate = year + month + day


cruiselon = specificstationrow['lon'].tolist() ##
cruiselat = specificstationrow['lat'].tolist() ##

cruiseCDOM320 = data['acdom_320'].tolist() ##
cruiseCDOM350 = data['acdom_350'].tolist() ##

RSFile = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Files/'+cruisedate+'*.nc')
while len(RSFile) == 0:
    cruisedate = str(int(cruisedate) + 1)
    RSFile = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Files/'+cruisedate+'*.nc')



timedifflist = [] ##


for rfile in RSFile:
    RSFiletime = int(rfile.split('/')[7][9:15])
    timediff = abs(RSFiletime - cruisetime[0])
    timedifflist.append(timediff)


ind = timedifflist.index(min(timedifflist))
actualRSFile = RSFile[ind]

RSFilekey = actualRSFile.split('/')[7][:15] #

originRSFile = glob('/home/hboi-ouri/Projects/RS_Files/MODISGrids/Cruise/*/AQUA_MODIS.'+RSFilekey+'.L2.OC.x.nc')
originalRSFile = originRSFile[0].split('/')[8] ##


with open('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+RSFilekey+'Adjusted_MODIS_L2.csv.csv', 'r') as file:
    data = pd.read_csv(file)
    RSlat = data['lat'].tolist()
    RSlon = data['lon'].tolist()

RSaCDOM300 = []
RSaCDOM355 = []

Cruisecoords = (cruiselat[0], cruiselon[0])
distances = []
for RScoords in zip(RSlat, RSlon):
    distances.append(abs(great_circle(Cruisecoords, RScoords).km))

ind = distances.index(min(distances))

with open('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+RSFilekey+'Adjusted_MODIS_L2.csv.csv', 'r') as file:
    data = pd.read_csv(file)
    row = data.iloc[ind,:]
    #Appending the previously created list of values for validation table column
    RSaCDOM300.append(row['aCDOM300'])
    RSaCDOM355.append(row['aCDOM355'])





top = len(cruiseCDOM350)
i = 0
while i < top:




    rowtoadd = [cruisefile, cruisedate, cruiselat[0], cruiselon[0], cruisedepth, cruiseCDOM300[i], cruiseCDOM355[i], 
                originalRSFile, RSFilekey, RSlat[0], RSlon[0], RSaCDOM300[0], RSaCDOM355[0], 
                min(timedifflist), min(distances), float(RSaCDOM300[0])-float(cruiseCDOM320[i]), 
                float(RSaCDOM355[0])-float(cruiseCDOM350[i])]
    with open('/home/hboi-ouri/Projects/NASA_ProjectExp/outputs/Tasks/Task8/Validation Table Surface Water Redo.csv', 'a') as file:
        writer_object = writer(file)
        writer_object.writerow(rowtoadd)
        file.close()

    i = i+1

NameError: name 'cruisefile' is not defined

In [ ]:
#### For CTD Data

files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/CTD CSV/CTD_GOM21FA*')

for eachfile in files:


    # SP22files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM22SP_*')
    # SU22files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM22SU_*')

    ### For getting CSV ID, date, and time

    data = pd.read_csv(eachfile)

    #############################Here, creating a list of values for validation table column
    cruisefile = eachfile.split('/')[8]
    cruiselon = data['longitude'].tolist()
    cruiselat = data['latitude'].tolist()
    cruisepressure = data['Pressure [dbar]'].tolist()
    cruiseCDOM = data['wetCDOM'].tolist()
    #############################



    ID = float(eachfile.split('_')[4])
    #
    datefile = pd.read_csv('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/CSV Dates.csv')

    row = datefile.loc[datefile['ID'] == ID]
    ##
    cdate = str(row['Date'])
    cdate = cdate.split('    ')[1]
    #
    cmonth = cdate[0:2]
    cday = cdate[3:5]
    cyear = '20' + cdate[6:8]
    ##
    ctime = str(row['Time'])
    #
    ctime = ctime.split('    ')[1]
    chour = ctime[0:2]
    cminute = ctime[3:5]
    csecond = ctime[6:8]

    ctime = int(chour+cminute+csecond)
    ##
    ###Finding the relevant RS File
    RSFile = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Files/'+cyear+cmonth+cday+'*.nc')
    timedifflist = []

    for rfile in RSFile:
        RSFiletime = int(rfile.split('/')[7][9:15])
        timediff = abs(RSFiletime - ctime)
        timedifflist.append(timediff)


    ind = timedifflist.index(min(timedifflist))
    # RSFile[ind] chooses the temporally closest file to our in situ data


    timekey = RSFile[ind].split('/')[7][0:15]

    ######################HERE, creating a list of values for validation table column
    originRSFile = glob('/home/hboi-ouri/Projects/RS_Files/MODISGrids/Cruise/*/AQUA_MODIS.'+timekey+'.L2.OC.x.nc')
    originalRSFile = originRSFile[0].split('/')[8]
    ######################

    with open('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+timekey+'Adjusted_MODIS_L2.csv.csv', 'r') as file:
        data = pd.read_csv(file)
        RSlatitude = data['lat'].tolist()
        RSlongitude = data['lon'].tolist()

    with open(eachfile, 'r') as file:
        data = pd.read_csv(file)
        Cruiselatitude = data['latitude'].tolist()
        Cruiselongitude = data['longitude'].tolist()    


    ################# HERE, creating a list of values for validation table column
    RSaCDOM300 = []
    RSaCDOM355 = []
    RSS275_295 = []
    RSDOC = []
    RSlat = []
    RSlon = []

    for Cruisecoords in zip(Cruiselatitude, Cruiselongitude):
        Cruisecoords = Cruisecoords
        distances = []
        for RScoords in zip(RSlatitude, RSlongitude):
            distances.append(abs(great_circle(Cruisecoords, RScoords).km))

        ind = distances.index(min(distances))


        with open('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+timekey+'Adjusted_MODIS_L2.csv.csv', 'r') as file:
            data = pd.read_csv(file)
            row = data.iloc[ind,:]
            #Appending the previously created list of values for validation table column
            RSaCDOM300.append(row['aCDOM300'])
            RSaCDOM355.append(row['aCDOM355'])
            RSS275_295.append(row['S275_295'])
            RSDOC.append(row['DOC'])
            RSlat.append(row['lat'])
            RSlon.append(row['lon'])







    #Validation Table:  /home/hboi-ouri/Projects/NASA_ProjectExp/outputs/Tasks/Task8/Validation Table.csv
    # Columns: Cruise Data, Cruise Time, Cruise Lat, Cruise Lon, Cruise CDOM, RS Data, RS Time, RS Lat, RS Lon, 
    #          RS aCDOM300, RS aCDOM355, RS S275_295, RS DOC, RS Time Diff, RS Distance Diff, RS aCDOM300 Diff, 
    #          RS aCDOM355 Diff, RS S275_295 Diff, RS DOC Diff






    top = len(cruiseCDOM)
    i = 0
    while i < top:
        rowtoadd = [cruisefile, ctime, cruiselat[i], cruiselon[i], cruisepressure[i], cruiseCDOM[i], originalRSFile, timekey, 
                    RSlat[i], RSlon[i], RSaCDOM300[i], RSaCDOM355[i], RSS275_295[i], RSDOC[i], min(timedifflist),
                    distances[i], float(RSaCDOM300[i])-float(cruiseCDOM[i]), 
                    float(RSaCDOM355[i])-float(cruiseCDOM[i]),
                    float(RSS275_295[i])-float(cruiseCDOM[i]), 
                    float(RSDOC[i])-float(cruiseCDOM[i])]
        with open('/home/hboi-ouri/Projects/NASA_ProjectExp/outputs/Tasks/Task8/Validation Table Test.csv', 'a') as file:
            writer_object = writer(file)
            writer_object.writerow(rowtoadd)
            file.close()
        i = i+1
    

In [3]:
FA21files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/CTD CSV/CTD_GOM21FA*')
files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/CTD CSV/CTD_GOM21FA*')
# SP22files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM22SP_*')
# SU22files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM22SU_*')

### For getting CSV ID, date, and time

data = pd.read_csv(files[0])

#############################Here, creating a list of values for validation table column
cruisefile = files[0]
cruiselon = data['longitude'].tolist()
cruiselat = data['latitude'].tolist()
cruiseCDOM = data['wetCDOM'].tolist()
#############################


ntime = files[0].split('/')[8]
ID = float(ntime.split('_')[2][2:6])
#
datefile = pd.read_csv('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/CSV Dates.csv')

row = datefile.loc[datefile['ID'] == ID]
##
cdate = str(row['Date'])
#
cmonth = cdate[5:7]
cday = cdate[8:10]
cyear = '20' + cdate[11:13]
##
ctime = str(row['Time'])
#
chour = ctime[5:7]
cminute = ctime[8:10]
csecond = ctime[11:13]
ctime = int(chour+cminute+csecond)
##

###Finding the relevant RS File
RSFile = glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Files/'+cyear+cmonth+cday+'*.nc')
timedifflist = []

for rfile in RSFile:
    RSFiletime = int(rfile.split('/')[7][9:15])
    timediff = abs(RSFiletime - ctime)
    timedifflist.append(timediff)
    
ind = timedifflist.index(min(timedifflist))
# RSFile[ind] chooses the temporally closest file to our in situ data


timekey = RSFile[ind].split('/')[7][0:15]

######################HERE, creating a list of values for validation table column
originRSFile = glob('/home/hboi-ouri/Projects/RS_Files/MODISGrids/Cruise/*/AQUA_MODIS.'+timekey+'.L2.OC.x.nc')
originalRSFile = originRSFile[0].split('/')[8]
######################

with open('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+timekey+'Adjusted_MODIS_L2.csv.csv', 'r') as file:
    data = pd.read_csv(file)
    RSlatitude = data['lat'].tolist()
    RSlongitude = data['lon'].tolist()
    
with open('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM21FA_ID2144_St6_10-25-21.csv', 'r') as file:
    data = pd.read_csv(file)
    Cruiselatitude = data['latitude'].tolist()
    Cruiselongitude = data['longitude'].tolist()    
    

################# HERE, creating a list of values for validation table column
RSaCDOM300 = []
RSaCDOM355 = []
RSS275_295 = []
RSDOC = []
RSlat = []
RSlon = []

for Cruisecoords in zip(Cruiselatitude, Cruiselongitude):
    Cruisecoords = Cruisecoords
    distances = []
    for RScoords in zip(RSlatitude, RSlongitude):
        distances.append(abs(great_circle(Cruisecoords, RScoords).km))

    ind = distances.index(min(distances))
    
    
    with open('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+timekey+'Adjusted_MODIS_L2.csv.csv', 'r') as file:
        data = pd.read_csv(file)
        row = data.iloc[ind,:]
        #Appending the previously created list of values for validation table column
        RSaCDOM300.append(row['aCDOM300'])
        RSaCDOM355.append(row['aCDOM355'])
        RSS275_295.append(row['S275_295'])
        RSDOC.append(row['DOC'])
        RSlat.append(row['lat'])
        RSlon.append(row['lon'])
              
        
    




#Validation Table:  /home/hboi-ouri/Projects/NASA_ProjectExp/outputs/Tasks/Task8/Validation Table.csv
# Columns: Cruise Data, Cruise Time, Cruise Lat, Cruise Lon, Cruise CDOM, RS Data, RS Time, RS Lat, RS Lon, 
#          RS aCDOM300, RS aCDOM355, RS S275_295, RS DOC, RS Time Diff, RS Distance Diff, RS aCDOM300 Diff, 
#          RS aCDOM355 Diff, RS S275_295 Diff, RS DOC Diff






top = len(cruiseCDOM)
i = 0
while i < top:
    rowtoadd = [cruisefile, ctime, cruiselat[i], cruiselon[i], cruiseCDOM[i], originalRSFile, timekey, 
                RSlat[i], RSlon[i], RSaCDOM300[i], RSaCDOM355[i], RSS275_295[i], RSDOC[i], min(timedifflist),
                distances[i], float(RSaCDOM300[i])-float(cruiseCDOM[i]), 
                float(RSaCDOM355[i])-float(cruiseCDOM[i]),
                float(RSS275_295[i])-float(cruiseCDOM[i]), 
                float(RSDOC[i])-float(cruiseCDOM[i])]
    with open('/home/hboi-ouri/Projects/NASA_ProjectExp/outputs/Tasks/Task8/Validation Table.csv', 'a') as file:
        writer_object = writer(file)
        writer_object.writerow(rowtoadd)
        file.close()
    i = i+1
    

IndexError: list index out of range

In [132]:
FA21files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM21FA_*')
files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM21FA_*')
# SP22files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM22SP_*')
# SU22files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM22SU_*')

### For getting CSV ID, date, and time

data = pd.read_csv(files[0])
print(files[0])

/home/hboi-ouri/Projects/RS_Files/Task8/CTD Data/Rosette CSV/ROSETTE_GOM21FA_ID2150_St9_11-01-21.csv


In [92]:
###Getting all RS files with CDOM, and converting to CSV, then overwriting with data not containing NaN
#Both this and the cell below are one-time use. Pointless to use in a loop, since it manipulates the files once

for file in glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Files/*_MODIS_L2.nc'):
    tag = file.split('/')[7][:15]
    ds = xr.open_dataset(file)
    ds = ds.to_dataframe()
    ds.to_csv('/home/hboi-ouri/Projects/RS_Files/Task8/Variable CSVs/'+ tag +'_MODIS_L2.csv')

In [106]:
###Taking all previously made RS CDOM CSVs, and overwriting with only data not containing NaN

for file in glob('/home/hboi-ouri/Projects/RS_Files/Task8/Variable CSVs/*_MODIS_L2.csv'):
    tag = file.split('/')[7][:15]
    with open('/home/hboi-ouri/Projects/RS_Files/Task8/Variable CSVs/'+ tag +'_MODIS_L2.csv', 'r') as file:
        data = pd.read_csv(file)
        data = data[data['aCDOM300'].notna()]
        data.to_csv('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Modded CSVs/'+ tag +'Adjusted_MODIS_L2.csv.csv')
